<a href="https://colab.research.google.com/github/Crossme0809/langchain-tutorials/blob/main/Open_Source_Vector_Database_ChromaDB_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
!pip install chromadb -q
!pip install sentence-transformers -q

Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 5, in <module>
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main.py", line 10, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/build_env.py", line 19, in <module>
    from pip._internal.cli.spinners import open_spinner
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/spinners.py", line 9, in <module>
    from pip._internal.utils.logging import get_indentation
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/utils/logging.py", line 13, in <module>
    from

In [64]:
import chromadb

client = chromadb.Client()

collection = client.create_collection("chromadb_demo")

默认情况下，Chroma使用Sentence Transformers all-MiniLM-L6-v2模型来创建嵌入。

In [65]:
collection.add(
    documents=["This is a document about cat", "This is a document about car"],
    metadatas=[{"category": "animal"}, {"category": "vehicle"}],
    ids=["id1", "id2"]
)

In [62]:
client.delete_collection("chromadb_demo")
client.list_collections()
print(client)

In [66]:
results = collection.query(
    query_texts=["dog"],
    n_results=1
)
results

{'ids': [['id1']],
 'embeddings': None,
 'documents': [['This is a document about cat']],
 'metadatas': [[{'category': 'animal'}]],
 'distances': [[1.336723804473877]]}

In [67]:
import os

def read_files_from_folder(folder_path):
    file_data = []

    for file_name in os.listdir(folder_path):
        if file_name.endswith(".txt"):
            with open(os.path.join(folder_path, file_name), 'r') as file:
                content = file.read()
                file_data.append({"file_name": file_name, "content": content})

    return file_data

folder_path = "pets"  # your folder path
file_data = read_files_from_folder(folder_path)

for data in file_data:
    print(f"File Name: {data['file_name']}")
    print(f"Content: {data['content']}\n")

File Name: Different Types of Pet Animals.txt
Content: Pet animals come in all shapes and sizes, each suited to different lifestyles and home environments. Dogs and cats are the most common, known for their companionship and unique personalities. Small mammals like hamsters, guinea pigs, and rabbits are often chosen for their low maintenance needs. Birds offer beauty and song, and reptiles like turtles and lizards can make intriguing pets. Even fish, with their calming presence, can be wonderful pets.

File Name: Nutrition Needs of Pet Animals.txt
Content: Proper nutrition is vital for the health and wellbeing of pets. Dogs and cats require a balanced diet that includes proteins, carbohydrates, and fats. Some may even have specific dietary needs based on their breed or age. Birds typically thrive on a diet of seeds, fruits, and vegetables, while reptiles have diverse diets ranging from live insects to fresh produce. Fish diets depend greatly on the species, with some needing live food 

In [68]:
documents = []
metadatas = []
ids = []
for index,data in enumerate(file_data):
  documents.append(data['content'])
  metadatas.append({'source':data['file_name']})
  ids.append(str(index+1))

In [ ]:
metadatas

[{'source': 'Different Types of Pet Animals.txt'},
 {'source': 'Nutrition Needs of Pet Animals.txt'},
 {'source': 'Health Care for Pets.txt'},
 {'source': 'Training and Behaviour of Pets.txt'},
 {'source': 'The Emotional Bond Between Humans and Pets.txt'}]

In [69]:
from chromadb.config import Settings
client = chromadb.Client(Settings(
    chroma_db_impl="duckdb+parquet",
    persist_directory="pet_db" # Optional, defaults to .chromadb/ in the current directory
))

In [70]:
pet_collection = client.create_collection("pet_collection")
pet_collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

In [71]:
results = pet_collection.query(
    query_texts=["What are the different kinds of pets people commonly own?"],
    n_results=1
)
results

{'ids': [['1']],
 'embeddings': None,
 'documents': [['Pet animals come in all shapes and sizes, each suited to different lifestyles and home environments. Dogs and cats are the most common, known for their companionship and unique personalities. Small mammals like hamsters, guinea pigs, and rabbits are often chosen for their low maintenance needs. Birds offer beauty and song, and reptiles like turtles and lizards can make intriguing pets. Even fish, with their calming presence, can be wonderful pets.']],
 'metadatas': [[{'source': 'Different Types of Pet Animals.txt'}]],
 'distances': [[0.7325009703636169]]}

In [72]:
results = pet_collection.query(
    query_texts=["What are the emotional benefits of owning a pet?"],
    n_results=1,
    where_document={"$contains":"reptiles"}
)
results

{'ids': [['1']],
 'embeddings': None,
 'documents': [['Pet animals come in all shapes and sizes, each suited to different lifestyles and home environments. Dogs and cats are the most common, known for their companionship and unique personalities. Small mammals like hamsters, guinea pigs, and rabbits are often chosen for their low maintenance needs. Birds offer beauty and song, and reptiles like turtles and lizards can make intriguing pets. Even fish, with their calming presence, can be wonderful pets.']],
 'metadatas': [[{'source': 'Different Types of Pet Animals.txt'}]],
 'distances': [[0.8378248810768127]]}

In [73]:
results = pet_collection.query(
    query_texts=["What are the emotional benefits of owning a pet?"],
    n_results=1,
    where={"source": "Training and Behaviour of Pets.txt"},
    # where_document={"$contains":"reptiles"}
)
results

{'ids': [['4']],
 'embeddings': None,
 'documents': [['Training is essential for a harmonious life with pets, particularly for dogs. It helps pets understand their boundaries and makes cohabitation easier for both pets and owners. Training should be based on positive reinforcement. Understanding pet behavior is also important, as changes in behavior can often be a sign of underlying health issues.']],
 'metadatas': [[{'source': 'Training and Behaviour of Pets.txt'}]],
 'distances': [[0.8881876468658447]]}

In [74]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L3-v2')

In [75]:
documents = []
embeddings = []
metadatas = []
ids = []
for index,data in enumerate(file_data):
  documents.append(data['content'])
  embeding = model.encode(data['content']).tolist()
  embeddings.append(embeding)
  metadatas.append({'source':data['file_name']})
  ids.append(str(index+1))

In [76]:
pet_collection_emb = client.create_collection("pet_collection_emb")
pet_collection_emb.add(
    documents=documents,
    embeddings=embeddings,
    metadatas=metadatas,
    ids=ids
)

In [77]:
query = "What are the different kinds of pets people commonly own?"
input_em = model.encode(query).tolist()

results = pet_collection_emb.query(
    query_embeddings=[input_em],
    n_results=1
)
results

{'ids': [['1']],
 'embeddings': None,
 'documents': [['Pet animals come in all shapes and sizes, each suited to different lifestyles and home environments. Dogs and cats are the most common, known for their companionship and unique personalities. Small mammals like hamsters, guinea pigs, and rabbits are often chosen for their low maintenance needs. Birds offer beauty and song, and reptiles like turtles and lizards can make intriguing pets. Even fish, with their calming presence, can be wonderful pets.']],
 'metadatas': [[{'source': 'Different Types of Pet Animals.txt'}]],
 'distances': [[12.040446281433105]]}

In [78]:
query = "foods that are recommended for  dogs?"
input_em = model.encode(query).tolist()

results = pet_collection_emb.query(
    query_embeddings=[input_em],
    n_results=1
)
results

{'ids': [['2']],
 'embeddings': None,
 'documents': [['Proper nutrition is vital for the health and wellbeing of pets. Dogs and cats require a balanced diet that includes proteins, carbohydrates, and fats. Some may even have specific dietary needs based on their breed or age. Birds typically thrive on a diet of seeds, fruits, and vegetables, while reptiles have diverse diets ranging from live insects to fresh produce. Fish diets depend greatly on the species, with some needing live food and others subsisting on flakes or pellets.']],
 'metadatas': [[{'source': 'Nutrition Needs of Pet Animals.txt'}]],
 'distances': [[17.143936157226562]]}

In [79]:
# pet_collection_emb.peek()

In [ ]:
# !rm -rf  '/content/pet_db'

In [80]:
!zip -r 'pet_db.zip' 'pet_db'

  adding: pet_db/ (stored 0%)
  adding: pet_db/index/ (stored 0%)
  adding: pet_db/index/uuid_to_id_2c1972f3-f9c9-4392-8db2-07d2b6d7c880.pkl (deflated 24%)
  adding: pet_db/index/id_to_uuid_2c1972f3-f9c9-4392-8db2-07d2b6d7c880.pkl (deflated 17%)
  adding: pet_db/index/index_metadata_c0da6928-2515-44bf-91ea-3e01ffc0e1af.pkl (deflated 14%)
  adding: pet_db/index/id_to_uuid_c0da6928-2515-44bf-91ea-3e01ffc0e1af.pkl (deflated 17%)
  adding: pet_db/index/index_2c1972f3-f9c9-4392-8db2-07d2b6d7c880.bin (deflated 14%)
  adding: pet_db/index/index_c0da6928-2515-44bf-91ea-3e01ffc0e1af.bin (deflated 15%)
  adding: pet_db/index/index_metadata_2c1972f3-f9c9-4392-8db2-07d2b6d7c880.pkl (deflated 15%)
  adding: pet_db/index/uuid_to_id_c0da6928-2515-44bf-91ea-3e01ffc0e1af.pkl (deflated 24%)


In [80]:
!pip install pypdf2

In [104]:
import os
from PyPDF2 import PdfReader

def read_pdf_files_from_folder(folder_path):
    file_data = []

    for file_name in os.listdir(folder_path):
        if file_name.endswith(".pdf"):
            with open(os.path.join(folder_path, file_name), 'rb') as file:
                reader = PdfReader(file)
                page = reader.pages[0]
                print(page.extract_text())
                information = reader.metadata
                file_data.append({"file_name": file_name, "content": page.extract_text()})

    return file_data

folder_path = "linux_doc"  # your folder path
pdf_data = read_pdf_files_from_folder(folder_path)

for data in pdf_data:
    print(f"File Name: {data['file_name']}")
    print(f"Content: {data['content']}\n")

Training language models to follow instructions
with human feedback
Long OuyangJeff WuXu JiangDiogo AlmeidaCarroll L. Wainwright
Pamela MishkinChong Zhang Sandhini Agarwal Katarina Slama Alex Ray
John Schulman Jacob Hilton Fraser Kelton Luke Miller Maddie Simens
Amanda AskellyPeter Welinder Paul Christianoy
Jan LeikeRyan Lowe
OpenAI
Abstract
Making language models bigger does not inherently make them better at following
a user’s intent. For example, large language models can generate outputs that
are untruthful, toxic, or simply not helpful to the user. In other words, these
models are not aligned with their users. In this paper, we show an avenue for
aligning language models with user intent on a wide range of tasks by ﬁne-tuning
with human feedback. Starting with a set of labeler-written prompts and prompts
submitted through the OpenAI API, we collect a dataset of labeler demonstrations
of the desired model behavior, which we use to ﬁne-tune GPT-3 using supervised
learning. 

In [105]:
documents = []
metadatas = []
ids = []
for index,data in enumerate(pdf_data):
  documents.append(data['content'])
  metadatas.append({'source':data['file_name']})
  ids.append(str(index+1))

In [106]:
from chromadb.config import Settings
client = chromadb.Client(Settings(
    chroma_db_impl="duckdb+parquet",
    persist_directory="linux_db" # Optional, defaults to .chromadb/ in the current directory
))

In [107]:
linux_collection = client.create_collection("linux_collection")
linux_collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

In [108]:
results = linux_collection.query(
    query_texts=["CentOS怎么查看当前时间?"],
    n_results=1
)
results

{'ids': [['2']],
 'embeddings': None,
 'documents': [['CentOS 常用命令\xa0\n什么是LNMP？就是Linux+Nginx+MySQL+PHP。\xa0\nCentOS 常用命令在我们的使用中经常被使用。所以对一些经常使用又很重要的 CentOS 常用命令进行了下面的收集。这里简单介绍这\n些 CentOS 常用命令。\x7f\n一：使用CentOS常用命令查看cpu \x7f\nmore\x7f/proc/cpuinfo\x7f|\x7fgrep\x7f“model\x7fname”\x7f\ngrep\x7f“model\x7fname”\x7f/proc/cpuinfo\x7f\n[root@localhost\x7f/]#\x7fgrep\x7f“CPU”\x7f/proc/cpuinfo\x7f\nmodel\x7fname\x7f:\x7fIntel(R)\x7fPentiu m(R)\x7fDual\x7fCPU\x7fE2180\x7f@\x7f2.00GHz\x7f\nmodel\x7fname\x7f:\x7fIntel(R)\x7fPentiu m(R)\x7fDual\x7fCPU\x7fE2180\x7f@\x7f2.00GHz\x7f\n如果觉得需要看的更加舒服\x7f\ngrep\x7f“model\x7fname”\x7f/proc/cpuinfo\x7f|\x7fcut\x7f-f2\x7f-d:\x7f\n二：使用CentOS常用命令查看内存\x7f\ngrep\x7fMemTotal\x7f/proc/meminfo\x7f\ngrep\x7fMemTotal\x7f/proc/meminfo\x7f|\x7fcut\x7f-f2\x7f-d:\x7f\nfree\x7f-m\x7f|grep\x7f“Mem”\x7f|\x7fawk\x7f‘{print\x7f$2}’\x7f\n三：使用CentOS常用命令查看cpu是32 位还是64 位\xa0 \x7f\n查看 CPU 位数(32\x7for\x7f64)\x7f\ngetconf\x7fLONG_BIT\x7f\n四：使用CentOS常用命令查看当前linux 的版本\xa0 \x7f\nmore\x7f/

In [109]:
results = linux_collection.query(
    query_texts=["Implications for alignment research"],
    n_results=1
)
results

{'ids': [['1']],
 'embeddings': None,
 'documents': [['Training language models to follow instructions\nwith human feedback\nLong Ouyang\x03Jeff Wu\x03Xu Jiang\x03Diogo Almeida\x03Carroll L. Wainwright\x03\nPamela Mishkin\x03Chong Zhang Sandhini Agarwal Katarina Slama Alex Ray\nJohn Schulman Jacob Hilton Fraser Kelton Luke Miller Maddie Simens\nAmanda AskellyPeter Welinder Paul Christiano\x03y\nJan Leike\x03Ryan Lowe\x03\nOpenAI\nAbstract\nMaking language models bigger does not inherently make them better at following\na user’s intent. For example, large language models can generate outputs that\nare untruthful, toxic, or simply not helpful to the user. In other words, these\nmodels are not aligned with their users. In this paper, we show an avenue for\naligning language models with user intent on a wide range of tasks by ﬁne-tuning\nwith human feedback. Starting with a set of labeler-written prompts and prompts\nsubmitted through the OpenAI API, we collect a dataset of labeler demonst